In [291]:
url = 'https://www.kaggle.com/sherinclaudia/sarcastic-comments-on-reddit'

### Supervised Learning Capstone  
#### You're ready to put into practice everything you've learned so far.  

####  First: Go out and find a dataset of interest. It could be from one of our recommended resources, some other aggregation, or scraped yourself. Just make sure it has lots of variables in it, including an outcome of interest to you.  

####  Second: Explore the data. Get to know the data. Spend a lot of time going over its quirks and peccadilloes. You should understand how it was gathered, what's in it, and what the variables look like.  

####  Third: Model your outcome of interest. You should try several different approaches and really work to tune a variety of models before using the model evaluation techniques to choose what you consider to be the best performer. Make sure to think about explanatory versus predictive power and experiment with both.  

####  So, here is the deliverable: Prepare a slide deck and 15 minute presentation that guides viewers through your model. Be sure to cover a few specific things:  

1. A specified research question your model addresses  
1. How you chose your model specification and what alternatives you compared it to  
1. The practical uses of your model for an audience of interest  
1. Any weak points or shortcomings of your model  


This presentation is not a drill. You'll be presenting this slide deck live to a group as the culmination of your work in the last two supervised learning units. As a secondary matter, your slides and / or the Jupyter notebook you use or adapt them into should be worthy of inclusion as examples of your work product when applying to jobs.

Good luck!

### Questions that need answering:

 1. What question are you trying to solve (or prove wrong) ?   
 __We are determining if a post was sarcastic.__
 1. What kind of data do you have? -> describe the source.. 
 __The dataset includes 1.3 million sarcastic comments from Reddit.  The data includes balanced and unbalanced   __
 1. Do some EDA, plots
 1. What's missing from the data and how do you deal with it?  
 __stuff.    __
 1. Where are the outliers and why should we pay attention to them?  
 __stuff   ...__
 1. How can you add, change, or remove features to get more out of your data?  
 __Feature importances, PCA, dropping low variance items, correlated feature pairs, features too highly correlated, feature engineering, transforming, timestamp and make a month, day, year column from it.__

In [292]:
# Constants
max_iterations         = 10            # set it to > 0 for determining the features inportance
random_state           = 57
rows_in_training_set   = 10000
rows_in_test_set       = 200000
test_size              = 0.10
train_size             = 0.90
rfc_test_size          = 50000
rfc_train_size         = 5000
run_CountVectorizer    = False
run_TfidfVectorizer    = True
BegTimeStampNewlines   = 3
EndTimeStampNewlines   = 3
EndTimeStamp           = '\n'*EndTimeStampNewlines+'End'
BegTimeStamp           = 'End'+'\n'*BegTimeStampNewlines

In [293]:
# Controls
flag_to_run_rf = False
flag_to_plot_them = False
flag_to_run_correlation_matrix = False
flag_to_run_features_importance = False
flag_to_run_gradient_boosting  = False
flag_to_run_linear_regression  = False
flag_to_run_logistic_regression = False
flag_to_run_lasso_regression = False
flag_to_run_ridge_regression = False
flag_to_run_svc = False
flag_to_run_vectorizer_nb = True

In [294]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

%matplotlib inline

import chardet
import datetime
from sklearn import ensemble
from sklearn import datasets
from sklearn import metrics
from sklearn import linear_model
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time, sys
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [295]:
# add this to a dictionary
# Constants
max_iterations         = 10            # set it to > 0 for determining the features inportance
random_state           = 57
test_size              = 0.10
train_size             = 0.90

begin_string = '\n'*3+'Begin'
end_string = 'End'+'\n'*3

# Regression/Classification control
Regression = False 

print("Regression = {}".format(Regression))

Regression = False


In [296]:
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [297]:
file = 'Sarcasm_Headlines_Dataset.json'
path = path="../../../../Datafiles/"
column_names = []

with open(path+file, 'rb') as f:
    result = chardet.detect(f.read()) # or readline if the file is large
# df=pd.read_csv(path+file,encoding=result['encoding'],header=None, names = column_names)
df = pd.read_json(path+file, lines=True)
print("there are {:,} entries in the data file {}".format(len(df), file))
print("columns={}".format(df.columns))

there are 26,709 entries in the data file Sarcasm_Headlines_Dataset.json
columns=Index(['article_link', 'headline', 'is_sarcastic'], dtype='object')


In [298]:
# data Cleanup
columns_to_cleanup = []
for column in columns_to_cleanup:
    print("we are now cleaning up column {}".format(column))
    df[column].fillna(-1, inplace=True)
# df['BareNuclei'] = np.where(df['BareNuclei'] == '?',-1, df['BareNuclei'])
# df['Class'] = np.where(df['Class'] == 4, 1, 0)
df['message'] = df['headline']
df['sentiment_label'] = df['is_sarcastic']

# convert to a numerical variable.  
# set only the 4's and 5's to 1, which is positive, and all others to 0.
# df['sentiment_label'] = df.sentiment.map({4:1, 5: 1, 3:0, 2: 0, 1:0})
print('we have cleaned up the dataframe.')

we have cleaned up the dataframe.


In [299]:
def print_timestamp_old(displaytext):    
    import sys
    import datetime
    datetime_now = str(datetime.datetime.now())
    print("{:19.19}: {} ".format(datetime_now, displaytext))

In [300]:
def print_timestamp(displaytext):    
    import sys
    import datetime
    datetime_now = str(datetime.datetime.now())
    print("{:19.19}: In: {} {} ".format(datetime_now, sys._getframe(1).f_code.co_name, displaytext))

In [301]:
def printFormatted(string):
    display(Markdown(string))

In [302]:
df.isnull().sum()

article_link       0
headline           0
is_sarcastic       0
message            0
sentiment_label    0
dtype: int64

In [303]:
df.columns
df.head(20)

,article_link,headline,is_sarcastic,message,sentiment_label
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0,j.k. rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world's women,0,advancing the world's women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab-grown meat,0,the fascinating case for eating lab-grown meat,0
7,https://www.huffingtonpost.com/entry/boxed-col...,"this ceo will send your kids to school, if you...",0,"this ceo will send your kids to school, if you...",0
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1,top snake handler leaves sinking huckabee camp...,1
9,https://www.huffingtonpost.com/entry/fridays-m...,friday's morning email: inside trump's presser...,0,friday's morning email: inside trump's presser...,0


In [304]:
# Definine outcome and predictors.
print("the current time of start is {}".format(str(datetime.datetime.now())))

# Make the categorical variables below into enumerated categorical variables
for dummy_column in []: # Remove these for now...
    df = pd.concat([df, pd.get_dummies(df[dummy_column])], axis=1)
df.columns = df.columns.str.replace(' ', '')

# y = df['Class']
# columns_excluded = ['Class','SampleCodeNumber'] # skip Class, the label, and SampleCodeNumber, which we do not need
# X = df.loc[:, ~df.columns.isin(columns_excluded)]

# print("End is {}".format(str(datetime.datetime.now())))

the current time of start is 2019-06-09 23:18:24.962331


In [305]:
# Definine outcome and predictors.
print("the current time of start is {}".format(str(datetime.datetime.now())))

y = df['sentiment_label']
X = df['message']

the current time of start is 2019-06-09 23:18:24.968567


In [306]:
X.sample(20)
X.head()

0    former versace store clerk sues over secret 'b...
1    the 'roseanne' revival catches up to our thorn...
2    mom starting to fear son's web series closest ...
3    boehner just wants wife to listen, not come up...
4    j.k. rowling wishes snape happy birthday in th...
Name: message, dtype: object

In [307]:
X.dtypes
X.describe()

count              26709
unique             26602
top       sunday roundup
freq                  10
Name: message, dtype: object

In [308]:
X.isnull().sum()

0

In [309]:
X.dtypes

dtype('O')

In [310]:
def plot_them():
    for column in X_train.columns:
#         plt.hist(X_train[column]*100, bins=40)
        plt.scatter(y_train, X_train[column]*100)
        plt.xlabel(column)
        plt.show()

In [311]:
def rfc_and_feature_importances(rf):    # Here we are using Random Forest classifier method to determine the top 30 features.

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size)
    
    ## Fit the model on your training data.
    rf.fit(X_train, y_train) 
    
    ## And score it on your testing data.
    rf.score(X_test, y_test)

    feature_importance = rf.feature_importances_

    # Make importances relative to max importance.
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    cols=X.columns[sorted_idx].tolist() 
    cols=cols[::-1]
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.subplot(1, 2, 2)
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, X.columns[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()
#     print("We are returning these columns {}".format(cols))
    return cols[:30] # return it sorted

In [312]:
def run_features_importance(rf,n):
# Here we will return the feature importances
    all_feature_important_columns = []
 
    for i in range(1,n):
        print_timestamp('running rfc iteration {} features importance for {} times'.format(i,n))
        columns2 = rfc_and_feature_importances(rf)
#         columns2.extend('{}'.format(i))
        all_feature_important_columns = all_feature_important_columns + columns2
    #     print("all_feature_import_columns={}".format(all_feature_important_columns))

    print("\nBOD:\nall_feature_important_columns = {}\nEOD".format(sorted(all_feature_important_columns)))
    for feature in set(all_feature_important_columns):
        print_timestamp("the NOC of feature {} in all_feature_important_columns is {}".format(feature, all_feature_important_columns.count(feature)))

In [313]:
def run_correlation_matrix():
    
    print_timestamp('Begin'+'\n'*3)
    
    # Setup the correlation matrix.
    corrmat = X.corr()
    print(corrmat)

    # Set up the subplots
    f, ax = plt.subplots(figsize=(12, 9))

    # Let's draw the heatmap using seaborn.
    sns.heatmap(corrmat, vmax=.6, square=True)
    plt.show()
    
    print_timestamp('\n'*3+'End')

In [314]:
# Let's fit it with the RFC training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=0)
print("train_size = {}, X_train is {}, and y_train is {}".format(train_size, len(X_train), len(y_train)))
print("test_size  = {}, X_test  is {}, and y_test is {}".format(test_size, len(X_test), len(y_test)))

train_size = 0.9, X_train is 24038, and y_train is 24038
test_size  = 0.1, X_test  is 2671, and y_test is 2671


In [315]:
def run_rf(rf):
    print_timestamp('Begin run_rf part 1')
    
    ## Fit the model on your training data.
    rf.fit(X_train, y_train)   
    
    ## Let's score it with the training data set
    train_score = rf.score(X_train, y_train)
    print("Training score = {}".format(train_score))

    ## Let's score it with the test data set
    test_score = rf.score(X_test, y_test)
    
    print("Test score = {}".format(test_score))
    #Let's run cross validate score with the training data set
#     cross_val_score(rf, X_train, y_train, cv=5)
    
    print_timestamp('End run_rfr part 1')

Let's try predicting with gradient boosting classification

In [316]:
def run_gradient_boosting():

    print_timestamp('Begin')
    
    clf = ensemble.GradientBoostingClassifier(**params)

    #Let's run cross validate score with the training data set
    cross_val_score(clf, X_train, y_train, cv=5)

    loss_function = 'deviance' # could be exponential
    depth_value = 8
    params = {'n_estimators': 500,
              'max_depth': 8,
              'loss_function': loss_function,
              'max_leaf_nodes': depth_value, # 8 worked best...
              'min_samples_leaf': depth_value * 3
              ,'random_state' : random_state
             }

    clf.fit(X_train, y_train)

    predict_train = clf.predict(X_train)
    predict_test = clf.predict(X_test)
    
    print_timestamp('End')

In [317]:
def run_svc():

    print_timestamp('\n'*3+'Begin run_svc')
    
    # Let's do a linear Support Vector Classifier
    print_timestamp('Running SVC(kernel=linear')
    svm = SVC(kernel = 'linear')
    
    # Let's fit the training model
    print_timestamp('Running svm.fit')
    svm.fit(X_train, y_train)
    
    # Let's score the training set
    print_timestamp('Running svm.score for the training set')
    svm.score(X_train, y_train)
    
    # Let's score the test set
    print_timestamp('Running svm.fit for the test set')
    svm.score(X_test, y_test)

    print_timestamp('\n'*3+'End run_svc')

In [318]:
def run_logistic_regression():
    print_timestamp('\n'*3+'Begin')

    lr = LogisticRegression(C=1e20, solver='lbfgs', max_iter=1000)

    print_timestamp('Running lr.fit for the training set')
    lr.fit(X_train, y_train)
    
    print_timestamp('Running lr.fit for the training set')
    print('\nR-squared simple model training set yields:')
    print(lr.score(X_train, y_train))
    print("here comes the test set")
    lrscore = lr.score(X_test, y_test)
    printFormatted("###  Logistic Regression score={:.2%}".format(lrscore))
    
    print_timestamp('\n'*3+'End')

In [319]:
def run_linear_regression():

    print_timestamp('\n'*3+'Begin')

    regr = linear_model.LinearRegression()

    print_timestamp('Running regr.fit for the training set')
    regr.fit(X_train, y_train)
    
    print("\nCoeffecients: \n", regr.coef_)
    print("\nIntercept: \n", regr.intercept_)
    print("\nR-squared for training data set:")
    print(regr.score(X_train, y_train))
    
    print("\nR-squared for test data set:")
    print(regr.score(X_test, y_test))
    
    print_timestamp('End run_linear_regression.\n\n')
    
    print_timestamp('\n'*3+'End')

In [320]:
def run_ridge_regression():
    # Fitting a ridge regression model. Alpha is the regularization
    # parameter (usually called lambda). As alpha gets larger, parameter
    # shrinkage grows more pronounced. Note that by convention, the
    # intercept is not regularized. Since we standardized the data
    # earlier, the intercept should be equal to zero and can be dropped.
    print_timestamp('\n'*3+'Begin')
    
    ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False) 
    ridgeregr.fit(X_train, y_train)
    print(ridgeregr.score(X_train, y_train))

    print_timestamp('\n'*3+'End')

In [321]:
def vectorizer_nb(type_of_vectorizer):

    print_timestamp(BegTimeStamp)
    
    # 1. import and instantiate CountVectorizer (with the default parameters)

    # 2. instantiate CountVectorizer (vectorizer)

#     X = df.message
#     y = df.sentiment_label

    # split X and y into training and testing sets
    # by default, it splits 75% training and 25% test
    # random_state=1 for reproducibility
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    # 3. fit & transform
    if type_of_vectorizer == 'Count':
        print("We are running with CountVectorizer")
        vectorizer = CountVectorizer()
        vectorizer.fit(X_train)
        vectorizer_method = 'CountVectorizer'
    elif type_of_vectorizer == 'Tfidf':
        print("We are running with TfidfVectorizer")
        vectorizer = TfidfVectorizer()
        vectorizer.fit_transform(X_train)
        vectorizer_method = 'TfidfVectorizer'
    
    # 4. transform training data
    X_train_dtm = vectorizer.transform(X_train)

    # equivalently: combine fit and transform into a single step
    # this is faster and what most people would do
    X_train_dtm = vectorizer.fit_transform(X_train)

    # 4. transform testing data (using fitted vocabulary) into a document-term matrix
    X_test_dtm = vectorizer.transform(X_test)

    # 1. import

    # 2. instantiate a Multinomial Naive Bayes model
    nb = MultinomialNB()

    # 3. train the model 
    # using X_train_dtm (timing it with an IPython "magic command")

    nb.fit(X_train_dtm, y_train)
    
    
    # 4. make class predictions for X_test_dtm
    y_pred_class = nb.predict(X_test_dtm)

    # calculate accuracy of class predictions

    met_test_score = metrics.accuracy_score(y_test, y_pred_class)
    printFormatted('###  With {} vectorizer, the metrics accuracy score = {:.2%}'.format(vectorizer_method,
                                                                                         met_test_score))
    
    print_timestamp(EndTimeStamp)

In [322]:
def main():
    
    print_timestamp('\n'*3+'Begin')
    
    if Regression == True:
        print_timestamp("We are running with a Regression model")
    elif Regression == False:
        print_timestamp("We are running with a Classifier model")
    else:
        print_timestamp("We have failed to set the Regression variable")
        sys.exit(main())
        

    if flag_to_plot_them == True:
        plot_them()

    if flag_to_run_features_importance == True:
        
        number_of_features_to_consider = 50
        params = {'n_estimators': 100}

        if Regression == True:
            print_timestamp('We are running RandomForestRegressor')
            rf = ensemble.RandomForestRegressor(**params)
            
        else:
            print_timestamp('We are running RandomForestClassifier')
            rf = ensemble.RandomForestClassifier(**params)

        run_features_importance(rf, number_of_features_to_consider)

    if flag_to_run_correlation_matrix == True:
        run_correlation_matrix()

    if flag_to_run_rf == True:
        #     params = {}
        params = {'n_estimators': 100} 

        if Regression == True:
            rf = ensemble.RandomForestRegressor(**params)
            print_timestamp('We are running RandomForestRegressor')
        else:
            rf = ensemble.RandomForestClassifier(**params)
            print_timestamp('We are running RandomForestClassifier')

        run_rf(rf)

    if flag_to_run_gradient_boosting  == True:
        run_gradient_boosting()

    if flag_to_run_linear_regression  == True:
        run_linear_regression()

    if flag_to_run_logistic_regression == True:
        run_logistic_regression()

    if flag_to_run_svc == True:
        run_svc() 

    if flag_to_run_ridge_regression == True:
        run_ridge_regression()
        
    if flag_to_run_vectorizer_nb == True:
        for vectorizer_iterator in ['Count', 'Tfidf']:
            vectorizer_nb(vectorizer_iterator)

    print_timestamp('End'+'\n'*3)

In [323]:
main()

2019-06-09 23:18:25: In: main 


Begin 
2019-06-09 23:18:25: In: main We are running with a Classifier model 
2019-06-09 23:18:25: In: vectorizer_nb End


 
We are running with CountVectorizer


###  With CountVectorizer vectorizer, the metrics accuracy score = 85.70%

2019-06-09 23:18:25: In: vectorizer_nb 


End 
2019-06-09 23:18:25: In: vectorizer_nb End


 
We are running with TfidfVectorizer


###  With TfidfVectorizer vectorizer, the metrics accuracy score = 83.11%

2019-06-09 23:18:26: In: vectorizer_nb 


End 
2019-06-09 23:18:26: In: main End


 
